# Libraries

In [145]:
import os

import torch
import torch.nn as nn 
import torch.nn.functional as F 
import torch.optim as optim

from torch.utils.data import Dataset
from torch.utils.data import DataLoader 

from torchvision import datasets, transforms
from torchvision.io import read_image
import torchvision.utils as vutils

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
from PIL import Image 
import imageio
from einops import rearrange 
from tqdm import tqdm 

import wandb

%matplotlib inline

In [48]:
print("CUDA available:", torch.cuda.is_available())
print("torch version:", torch.__version__)


CUDA available: False
torch version: 2.6.0


In [185]:
'''
t1 = torch.randn((3, 3, 5, 5))
c1 = nn.Conv2d(in_channels=3, out_channels=5, kernel_size=2, stride=1)

flat = nn.Flatten(start_dim=1)
t2 = c1(t1)
print(t2.size())

flat_t2 = flat(t2)
print(flat_t2.size())
'''


'\nt1 = torch.randn((3, 3, 5, 5))\nc1 = nn.Conv2d(in_channels=3, out_channels=5, kernel_size=2, stride=1)\n\nflat = nn.Flatten(start_dim=1)\nt2 = c1(t1)\nprint(t2.size())\n\nflat_t2 = flat(t2)\nprint(flat_t2.size())\n'

# CVAE Model

### Creating the CVAE model.


In [198]:
#######################################################
# CVAE IMPLEMENTATION 
#######################################################

class CVAE(nn.Module): 
    def __init__(self, in_channel_1=3, out_channel_1=32, in_channel_2=32, out_channel_2 = 64): 
        super().__init__()

        # Encoder Layers: conv1 -> batchNorm1 -> ReLU1 -> conv2 -> batchNorm2 -> ReLU2 -> Flatten -> FC 
        self.c1 = nn.Conv2d(in_channel_1, out_channel_2, kernel_size=4)
        self.bn1 = nn.BatchNorm2d(out_channel_1)
        self.r1 = nn.ReLU()
        self.c2 = nn.Conv2d(in_channel_2, out_channel_2, kernel_size=4)
        self.bn2 = nn.BatchNorm2d(out_channel_2)
        self.r1 = nn.ReLU()
        self.flat = nn.Flatten(start_dim=1) # flatten everything (including color channels)
        
        
        #self.fc1 = nn.Linear()

        # Decoder Layers: FC -> unflatten -> convTranspose -> batchNorm -> ReLU -> output

    def __get_encoder_output_shape(self, x):
        """
        Runs a forward pass on the encoder portion so I can get the shape for nn.Linear
        """
        x = self.c1(x)
        x = self.bn1(x)
        x = self.r1(x) 
        x = self.c2(x)
        x = self.bn2(x)
        x = self.r1(x)
        x = self.flat(x) 
        return x
       
    def encode(self, x: torch.Tensor) -> tuple[torch.Tensor, torch.Tensor]:
        '''
        The encoder portion of the CVAE. 

        Args: 
            x (Tensor): Input data of shape [batch_size, 178 x 218], flattened images.

        Returns:
            Tuple[Tensor, Tensor]: The mean and log-variance of the approximate posterior 
            distribution q(z|x), where z is the latent variable. 
        '''

        h1 = F.relu(self.fc1(x)) 
        h2 = self.fc2(h1)
        h3 = F.relu(self.fc3(h2))
        mu = self.fc41(h3)
        logvar = self.fc42(h3) 
        
        return mu, logvar

    def reparametrize(self, mu: torch.Tensor, logvar: torch.Tensor) -> torch.Tensor:
        '''
        Reparametrization trick standard in CVAE's. 

        Args: 
            mu: The mean of the approximate posterior distribution: q(z|x)
            logvar: The log-variance of the approximate posterior distribution: q(z|x)
        
        Returns: 
            z: The latent variable sampled from q(z|x) using the reparametrization trick.
            z is size [batch_size, latent_dim], in this case, latent_dim=100.
        ''' 
        
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)

        z = mu + std * eps 
        return z 

    def decode(self, z: torch.Tensor) -> torch.Tensor:
        '''
        The decoder portion of CVAE. 
        Args: 
            z: The latent variable. 

        Returns: 
            x_recon: The reconstructed x sampled from the learned distribution of the decoder. 
            In this case, x_recon is a reconstructred picture. 
        '''
        h1 = F.relu(self.fc5(z))
        h2 = F.relu(self.fc6(h1))
        x_recon = F.sigmoid(self.fc7(h2))

        return x_recon
    
    def forward(self, x: torch.Tensor) -> tuple[torch.Tensor, torch.Tensor, torch.Tensor]: 
        """
        The forward pass on the CVAE. 
        """

        mu, logvar = self.encode(x)
        z = self.reparametrize(mu, logvar)
        x_recon = self.decode(z)
        
        return x_recon, mu, logvar

In [199]:
"""
CVAE TESTING CELL 
"""

# Tensor in (N, C, H, W)
x = torch.zeros([1, 3, 218, 178])   # Test "picture"
#print(test_x)
#print("Shape:", test_x.size())

cvae = CVAE() 

x_recon = cvae._CVAE__get_encoder_output_shape(x)
print(x_recon.size())



RuntimeError: running_mean should contain 64 elements not 32

# Preprocessing Step
### Preprocess the CelebA labels such that they are a one-hot encoded vector. 

After finishing the prototype, we will put this preprocessing part into a separate file in utils/preprocessing.py

In [112]:
def preprocess_CelebA_labels(input_path:str, output_path:str) -> None: 
    """
    Changes the CelebA labels to a one-hot encoding instead of using 1 and -1. 

    Args: 
        input_path: The labels csv to read in.
        output_path: Create a new csv into this path. 
    """

    label_df = pd.read_csv(input_path)
    label_df.iloc[:, 1:] = (label_df.iloc[:, 1:] + 1)//2 # converts to one-hot 
    label_df.to_csv(output_path, index=False)
    print(f"Processed labels saved into {output_path}.")

In [113]:
# Testing preprocessor  (WORKS!!

input = "../data/CelebA/list_attr_celeba.csv"
output = "../data/CelebA/preprocessed_labels.csv"
#preprocess_CelebA_labels(input, output)

# Dataset Class

### Create a custom dataset class compatible with the torch.DataLoader class
### https://pytorch.org/tutorials/beginner/basics/data_tutorial.html

The custom dataset class needs 3 functions: `__init__`, `__len__`, and `__getitem__`. 

Use an image directory, in my case "data/CelebA/img_align_celeba/. 

Use an annotations file, which is the metadata for the image, or the relevant features for the model to know. 

Notes: 
    - transforms perform manipulation on data to make it suitable for machine learning models to train on.

In [106]:
class CelebAImageDataset(Dataset):
    def __init__(self, img_dir, annotations_file, transform=None, target_transform=None):
        '''
        Args:
            annotations_file: The dataset containing all of the relevant labels. 
            img_dir: The path to the images folder. 
            transform: Modifies the feature data.
            target_transform: Modifies the label data.   
        '''
        
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir 
        self.transform = transform 
        self.target_transform = target_transform 

        #self.img_landmarks = pd.read_csv(landmarks_file) for simplicity, i wont use this yet.

    
    def __len__(self):
        return len(self.img_labels)
    
    def __getitem__(self, idx: int) -> tuple[torch.Tensor, pd.Series]: 
        '''
        Retrieves the image and its labels of the corresponding index. 
        '''
        # img_dir in this case is ../data/CelebA/img_align_celeba/img_align_celeba
        # ie. ../data/CelebA/img_align_celeba/img_align_celeba/000001.jpg
        
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0]) 
        image = read_image(img_path)
        labels = self.img_labels.iloc[idx, 1:] # retrieves the vector of labels
        labels = torch.tensor(labels) # converts pandas series obj to tensor obj
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            labels = self.target_transform(labels)

        return image, labels 

In [114]:
"""
Testing the dataset class
"""

annotations_file = "../data/CelebA/preprocessed_labels.csv"
img_dir = "../data/CelebA/img_align_celeba/img_align_celeba"

celeba = CelebAImageDataset(img_dir=img_dir, annotations_file=annotations_file)
image, labels = celeba.__getitem__(0) # retrieve image

#display(image)

/var/folders/89/s23t762151989wcy_1bx9vqc0000gn/T/ipykernel_36615/2372924361.py:32: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  labels = torch.tensor(labels) # converts pandas series obj to tensor obj


In [63]:
"""
A testing function to display images. 
"""

def display(image: torch.Tensor): 
    image_np = image.permute(1, 2, 0).numpy() # convert into numpy array with proper dims
    plt.imshow(image_np)
    plt.axis("off")
    plt.show()

# Dataset Manager Class 
### Responsible for partitioning and creating the dataloader. 

In [115]:
from torch.utils.data import Subset

class CelebADatasetManager:
    def __init__(self, img_dir, annotations_file, transform=None, target_transform=None):
        self.dataset = CelebAImageDataset(
            img_dir=img_dir,
            annotations_file=annotations_file,
            transform=transform,
            target_transform=target_transform
        )

        self.split_values = None

    def partition(self, split_value: float):
        """
        Partitions the dataset based on the split_value. ie. 0.6 means 60% train, 40% test.
        Args: 
            split_value: The value that splits the dataset.
        Returns: 
            tuple[torch.Subset, torch.Subset]: Returns the train and test dataset. 
        """
        if split_value >= 1 or split_value <= 0:
            assert False, "split_value must be between (0,1)"
        
        train_split = split_value 
        test_split = 1 - split_value

        train_idx = round(len(self.dataset) * train_split)
        test_idx = round(len(self.dataset) * test_split)

        train_dataset = Subset(self.dataset, np.arange(train_idx))
        test_dataset = Subset(self.dataset, np.arange(test_idx))

        return train_dataset, test_dataset
    

    def get_dataloader(self, split_value: float, batch_size: int = 64, shuffle:bool = True):
        """
        Creates train and test dataloaders. 

        Args: 
            split_value: The value to split the dataset. ie split_value=0.6 corresponds to 60% train and 40% test. 
            batch_size: The batch size when doing SGD.
            shuffle: Shuffles the dataset after every epoch. 
        
        Returns: 
            A tuple containing the train and test dataloaders.
        """
        train_dataset, test_dataset = self.partition(split_value)
        
        train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle)
        test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle= shuffle)
        
        return train_dataloader, test_dataloader




In [155]:
"""
Testing the dataset manager class.
"""

annotations_file = "../data/CelebA/preprocessed_labels.csv"
img_dir = "../data/CelebA/img_align_celeba/img_align_celeba"

ds_manager = CelebADatasetManager(img_dir=img_dir, annotations_file=annotations_file)

train_ds, _ = ds_manager.partition(0.6)
image, _ = train_ds[0]

train_dl, test_dl = ds_manager.get_dataloader(split_value=0.6)
train_features, train_labels = next(iter(train_dl))

print(train_features.size())
train_labels.size()

print(image.size())




torch.Size([64, 3, 218, 178])
torch.Size([3, 218, 178])


/var/folders/89/s23t762151989wcy_1bx9vqc0000gn/T/ipykernel_36615/2372924361.py:32: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  labels = torch.tensor(labels) # converts pandas series obj to tensor obj


# Loss Function
### Defines a loss function for the CVAE 

In [138]:
def celbo_loss(x_recon, x, mu, logvar, beta:float = 1.0) -> float:
    """
    The ELBO loss used in VAE's / CVAE's.

    Args:
        x_recon: The reconstructed x created from the CVAE 
        x: The original x. 
        mu: The mean of the latent distribution. 
        logvar: The logvariance of the latent distribution.
        beta: The hyperparameter to control how strong the KL loss is. 

    Returns: 
        The total loss (scalar). 
    """

    # computes the kl divergence per sample. 
    # for example, with a batch size of 4, we get: [kl_1, kl_2, kl_3, kl_4]
    kl = -1/2 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp(), dim=1) 
    kl = torch.mean(kl, dim=0) # average over batch size
 
    # take ce loss across samples and get the average across all samples 
    bce = F.binary_cross_entropy(x_recon, x, reduction="mean") 

    elbo_loss = beta * kl + bce
    return elbo_loss
    

# Training the Model
### I'm going to create a Trainer class to organize and create modular code for training.

In [168]:
class Trainer():
    def __init__(self, model, optimizer, train_loader, loss_fn, device = "cpu"):
        """
        Args:
            model: The model architecture the trainer will use. 
            optimizer: The optimizer the trainer will use (adam). 
            train_loader: The training dataset wrapped into the Pytorch DataLoader iterable object. 
            loss_fn: The loss function. 
            epochs: The number of epochs the model will train on.  
            device: The device that PyTorch will run calculations under.
        """
        self.model = model 
        self.optimizer = optimizer 
        self.train_loader = train_loader 
        self.loss_fn = loss_fn 
        self.device = device 
    
    def train(self, epochs=3): 
        """
        Trains the model. 
        """
        for batch_x, _ in self.train_loader: 
            print(batch_x.size())
            return None

In [169]:
cvae = CVAE()

annotations_file = "../data/CelebA/preprocessed_labels.csv"
img_dir = "../data/CelebA/img_align_celeba/img_align_celeba"

ds_manager = CelebADatasetManager(img_dir=img_dir, annotations_file=annotations_file)
train_dl, _ = ds_manager.get_dataloader(0.5)


opt = optim.Adam(cvae.parameters(), lr=1e-3)
trainer = Trainer(model=cvae, optimizer=opt, train_loader=train_dl, loss_fn=celbo_loss)
trainer.train(1)



torch.Size([64, 3, 218, 178])


/var/folders/89/s23t762151989wcy_1bx9vqc0000gn/T/ipykernel_36615/2372924361.py:32: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  labels = torch.tensor(labels) # converts pandas series obj to tensor obj
